# 内容
```meeting_0122_1_0```の特徴量固定版のはずが、下書き、トライアンドエラーノートに。

# したいこと
```meeting_0122_1_0```で発生した```/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide```の解明

# 方法
特徴量をある値に固定して動作を確認する


# 課題
```/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom```がまだ無くならない

# 考えたこと
この問題、そもそも回帰ではなく分類ではないか？？```X```が患者さんのいろんな特徴（yes/no形式）が書かれた患者さんデータで、```y```が罹患しているか否かだとしたら二値分類だ。回帰でしようとしていたからずっと```warning```が発生していたのだろうか？？

少なくとも今は、```X```も```y```も非連続数を使っていて、二値である、カテゴリとも言える変数を使っている。確認。

In [63]:
a = np.array([[2, 4, 4], [3, 5, 6]])
a.shape[0] #行数
a.mean(axis=0)

array([2.5, 4.5, 5. ])

## VarianceThreshold 

* 所定の分散を満たさないサンプルを持つ特徴を除去する
* デフォルトでは0分散つまり全てのサンプルが同じ値である特徴が除去される。他の例として, 0, 1のブール値を持つ特徴に対し、サンプルの80％以上が0または1の特徴を削除する例が記載されている。

In [ ]:
'''
XX = desc_df.values
from sklearn.feature_selection import VarianceThreshold
select = VarianceThreshold()
XX_new = select.fit_transform(XX)
'''

In [84]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, math, warnings
from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import f_regression
from sklearn.feature_selection import VarianceThreshold #所定の分散を満たさないサンプルを持つ特徴を除去する
#from sklearn.pipeline import Pipeline #何段にも渡る前処理や推論フェーズをわかりやすく表現する
from sklearn.feature_selection import chi2
import math
import itertools
import pprint

#variable
num_select = 2

In [85]:
# 説明変数（こっちの特徴量上位〇つ取り出す）
X = pd.read_table('training_markers16.txt', header=None,  sep ='\t')


data_yields = pd.read_table('training_yields16.txt', header=None,  sep ='\t')
#print(data_yields)

# 目的変数
y= data_yields.T.iloc[0]

In [38]:
def f_regression(X, y, center=True):
    """Univariate linear regression tests.
    Linear model for testing the individual effect of each of many regressors.
    This is a scoring function to be used in a feature seletion procedure, not
    a free standing feature selection procedure. #独立な特徴量の選択過程ではないよ
    
    This is done in 2 steps:
    1. The correlation(相関) between each regressor and the target is computed,
       that is, ((X[:, i] - mean(X[:, i])) * (y - mean_y)) / (std(X[:, i]) *
       std(y)).
    2. It is converted to an F score then to a p-value.
    For more on usage see the :ref:`User Guide <univariate_feature_selection>`.
    Parameters
    ----------
    X : {array-like, sparse matrix}  shape = (n_samples, n_features)
        The set of regressors that will be tested sequentially.
    y : array of shape(n_samples).
        The data matrix
    center : True, bool,
        If true, X and y will be centered.
    Returns
    -------
    F : array, shape=(n_features,)
        F values of features.
    pval : array, shape=(n_features,)
        p-values of F-scores.
    See also
    --------
    mutual_info_regression: Mutual information for a continuous target.
    f_classif: ANOVA F-value between label/feature for classification tasks.
    chi2: Chi-squared stats of non-negative features for classification tasks.
    SelectKBest: Select features based on the k highest scores.
    SelectFpr: Select features based on a false positive rate test.
    SelectFdr: Select features based on an estimated false discovery rate.
    SelectFwe: Select features based on family-wise error rate.
    SelectPercentile: Select features based on percentile of the highest
        scores.
    """
    # Xとyの長さが一致しているかどうかをチェックし、Xを2Dおよびyを1Dにする。
    X, y = check_X_y(X, y, ['csr', 'csc', 'coo'], dtype=np.float64)
    n_samples = X.shape[0] #m、行数、サンプル数

    # compute centered values
    # note that E[(x - mean(x))*(y - mean(y))] = E[x*(y - mean(y))], so we
    # need not center X
    if center: #今回Trueなのでこっち
        y = y - np.mean(y)
        if issparse(X):
            #Xを行方向に平均を取り、要素が列個分(n個)になったら順番に並べて一列にする。
            X_means = X.mean(axis=0).getA1() 
        else:
            #
            X_means = X.mean(axis=0)
        # compute the scaled standard deviations via moments
        
        # [row_norms(X.T, squared=True) - n_samples * X_means ** 2]の平方根
        # row_norms(X, squared=True)==np.sqrt((X * X).sum(axis=1))
        #X_norms==0を避けたい
        X_norms = np.sqrt(row_norms(X.T, squared=True) - n_samples * X_means ** 2) #
    else:
        # row_norms(X, squared=True)==np.sqrt((X * X).sum(axis=1))
        #X_norms==0を避けたい
        X_norms = row_norms(X.T) #

    # compute the correlation
    corr = safe_sparse_dot(y, X)#y, Xが疎行列であることが前提。returnされるのはret=y*Xまたはnp.dot(y, X)
    corr /= X_norms #X_norms==0だとアウト
    corr /= np.linalg.norm(y)#corrをyのベクトル長で割る

    # convert to p-value
    degrees_of_freedom = y.size - (2 if center else 1)#degrees_of_freedomはいつでも正だろう
    
    #corr==1のものは無視して、残りの特徴量で順位を以下で決めてもらう
    F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom #各特徴量ごとに値が出る
    
    #Fの値が大きいほどpvは小さくなるのかも
    pv = stats.f.sf(F, 1, degrees_of_freedom) #順番に並べる
    return F, pv

In [96]:
#X[:, 9]  #TypeError: '(slice(None, None, None), 9)' is an invalid key
X[:][9] 
y.size

16

<img src = '2020012113316.png' width = 70%>

In [103]:
#実はf_regressionでしたい計算は以下だけではないか、というのを検証👇
corr = 0
for i in range(X.shape[1]):
    corr = ((X[:][i] - np.mean(X[:][i])) * (y - np.mean(y))) / (np.std(X[:][i]) * np.std(y))
pprint.pprint(corr)
p = 1
degrees_of_freedom = (y.size-(p+1))/p
F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
    #16要素（行数と同じ）のが列ごと（16つ）出力された
    #これがcorretionとすると、
pprint.pprint(F)

from scipy import stats #scipy.stats.統計関数　例）正規分布なら，scipy.stats.norm
#fは分布関数として使われている
pv = stats.f.sf(F, 1, degrees_of_freedom) #scipy.stats.確率分布.sf()...生存関数。ある点以上の事象が起こる確率
pv

0    -0.774597
1    -0.774597
2     0.774597
3     0.774597
4    -0.774597
5     1.290994
6    -1.290994
7    -1.290994
8     1.290994
9    -0.774597
10    0.774597
11   -1.290994
12    1.290994
13   -0.774597
14    0.774597
15    0.774597
dtype: float64
0     21.0
1     21.0
2     21.0
3     21.0
4     21.0
5    -35.0
6    -35.0
7    -35.0
8    -35.0
9     21.0
10    21.0
11   -35.0
12   -35.0
13    21.0
14    21.0
15    21.0
dtype: float64


array([4.26375757e-04, 4.26375757e-04, 4.26375757e-04, 4.26375757e-04,
       4.26375757e-04, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 4.26375757e-04, 4.26375757e-04, 1.00000000e+00,
       1.00000000e+00, 4.26375757e-04, 4.26375757e-04, 4.26375757e-04])

In [94]:
d = np.array([[0, 1, 1], [1, 1, 0], [0, 0, 1]])
np.sqrt((d.T * d.T).sum(axis=1))

array([1.        , 1.41421356, 1.41421356])

In [74]:
d_means = d.mean(axis=0)
d.shape[0] * d_means ** 2

array([0.33333333, 1.33333333, 1.33333333])

In [77]:
d_norms = np.sqrt((d.T * d.T).sum(axis=1) - d.shape[0] * d_means ** 2)

In [83]:
e = np.array([0, 1, 0]).T
d_corr = e*d/d_norms/np.linalg.norm(e)
pprint.pprint(d_corr)
pprint.pprint(1 - corr ** 2)
pprint.pprint(corr ** 2 / (1 - corr ** 2))

array([[0.        , 1.22474487, 0.        ],
       [0.        , 1.22474487, 0.        ],
       [0.        , 0.        , 0.        ]])
array([0.85714286, 0.85714286, 0.98412698, 1.        , 0.98412698,
       0.98412698, 0.75      , 1.        , 1.        , 0.75      ,
       1.        , 1.        , 0.9375    , 1.        , 0.98412698,
       1.        ])
array([0.16666667, 0.16666667, 0.01612903, 0.        , 0.01612903,
       0.01612903, 0.33333333, 0.        , 0.        , 0.33333333,
       0.        , 0.        , 0.06666667, 0.        , 0.01612903,
       0.        ])


In [70]:
def safe_sparse_dot(a, b, dense_output=False):
    """Dot product that handle the sparse matrix case correctly"""
    from scipy import sparse
    if sparse.issparse(a) or sparse.issparse(b): #a, bいずれかが疎行列だったら
        ret = a * b
        #'numpy.ndarray' object has no attribute 'toarray'
        if dense_output and hasattr(ret, "toarray"): #Falseかつretが"toarray"という属性を持っているとき
            ret = ret.toarray() #retを密行列にする
        return ret
    else:
        return np.dot(a,b)
    
def safe_sqr(X):
    return X**2

In [65]:
#getA1()はlistにもnumpy.ndarrayにもseriesにも使えない
c = np.array([[2], [3], [5], [7]])
c.getA1()

AttributeError: 'numpy.ndarray' object has no attribute 'getA1'

In [40]:
b = np.matrix(np.arange(12).reshape((3,4)))
pprint.pprint(b)
b.getA1() #二次元行列を一次元ベクトルにする

matrix([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

<img src = '20200120114140.png'>

> modules that use this function such as graphnet will fail with the new scikit-learn version; it should now be imported from scipy.linalg

In [43]:
print('X.shape={}, y.shape={}'.format(X.shape, y.shape))
print('len(X)={}, len(y)={}'.format(len(X), len(y))) #check_X_yをpassするはずなんだが

X.shape=(16, 16), y.shape=(16,)
len(X)=16, len(y)=16


In [44]:
#エラー発生源調査


y = y - np.mean(y)
print(y)
#ある配列(ndarrayやリスト)をコピーして別のndarrayとして扱う 
#データをメモリ上にどのような順序で保持するかのモードがF
X = X.copy('F')
X -= X.mean(axis=0)
print(X)

# compute the correlation  
corr = safe_sparse_dot(y, X)#y, Xが疎行列であることが前提。returnされるのはret=y*Xまたはnp.dot(y, X)
print(corr)
corr /= np.asarray(np.sqrt(safe_sqr(X).sum(axis=0))).ravel()#((Xの2乗)を行ごとにsum)をコピーしたもので割る   
corr /= norm(y) #corrをyのベクトル長で割る
print(corr)
print(corr ** 2)
print(1 - corr ** 2)

0     0.5
1     0.5
2    -0.5
3    -0.5
4     0.5
5     0.5
6    -0.5
7    -0.5
8     0.5
9     0.5
10   -0.5
11   -0.5
12    0.5
13    0.5
14   -0.5
15   -0.5
Name: 0, dtype: float64
        0       1       2    3       4       5    6    7    8    9    10   11  \
0  -0.4375  0.5625 -0.4375  0.5 -0.4375  0.4375 -0.5  0.5 -0.5 -0.5  0.5 -0.5   
1   0.5625  0.5625 -0.4375 -0.5  0.5625  0.4375 -0.5  0.5  0.5 -0.5 -0.5  0.5   
2  -0.4375 -0.4375 -0.4375  0.5 -0.4375  0.4375  0.5  0.5  0.5 -0.5  0.5  0.5   
3  -0.4375 -0.4375  0.5625 -0.5  0.5625 -0.5625  0.5  0.5 -0.5 -0.5  0.5 -0.5   
4   0.5625  0.5625  0.5625  0.5 -0.4375 -0.5625 -0.5 -0.5  0.5  0.5 -0.5 -0.5   
5   0.5625 -0.4375  0.5625 -0.5 -0.4375 -0.5625  0.5 -0.5  0.5  0.5 -0.5  0.5   
6  -0.4375  0.5625  0.5625  0.5  0.5625  0.4375 -0.5 -0.5  0.5  0.5  0.5 -0.5   
7   0.5625  0.5625 -0.4375 -0.5  0.5625  0.4375  0.5 -0.5 -0.5  0.5 -0.5  0.5   
8  -0.4375 -0.4375 -0.4375  0.5 -0.4375 -0.5625 -0.5  0.5 -0.5 -0.5  0.5 -0.5   
9   0.

In [45]:
print('len(y) = {}'.format(len(y)))
print('type(y) = {}'.format(type(y)))

len(y) = 16
type(y) = <class 'pandas.core.series.Series'>


In [ ]:
#特徴量の分散や同じ特徴量が存在するのかを確認

sel = VarianceThreshold(threshold=0)
sel.fit_transform(X)

# get_supportで保持するデータのみをTrue値、そうでないものはFalse値を返します
print(sum(sel.get_support())==X.shape[1]) #同じ特徴量は存在していないことがわかった

In [ ]:
def selection(num_select, X, y):
    # num_select個の特徴量を選択
    #print(f_regression(X,y))
    #print(np.linalg.norm(y))
    #
    selector_VT = VarianceThreshold()
    selector_VT.fit_transform(X, y)
    selector_SKB = SelectKBest(score_func=chi2, k=num_select) #SelectKBest #f_regression...線形回帰
    selector_SKB.fit(X, y)
    mask_each = selector_SKB.get_support() #各特徴量を選択したか否かのmaskを取得
    return mask_each

In [ ]:
for num_select in range(1, X.shape[1]):
    start = time.time()
    selection(num_select, X, y)
    e_time = time.time() - start
    print("{} s in needed to execute selection({}, X, y)".format(num_select, num_select))

In [ ]:
start = time.time()
# 計測したい処理
e_time = time.time() - start
print ("e_time:{0}".format(e_time) + "[s]")

In [110]:
def each_energy(mask_each, num_select): #毎回のmaskとyが引数、合致度合いの計算を行う
    standard_mask = selection(num_select, X, y)
    energy_trait_sum = 0
    for i in range(len(mask_each)):
        if mask_each[i]==standard_mask[i]:
            energy_trait_sum += 1
    return energy_trait_sum #もとのyのときのmask(これをmaskとしている)との合致度合い（大きい⇒よく合致している）

In [111]:
selected_X_energy = X.shape[1]#maskのlenなので

In [112]:
def y_renewal(s: int):
    return 1-s

In [51]:
print('yの全部の組み合わせは{}通り'.format(2**16))
#だけど全部１と全部０を後で除かないといけない気がする

yの全部の組み合わせは65536通り


In [52]:
r = range(2) #0, 1なので
y_all_combination = list(itertools.product(r, repeat=16))
y_all_combination[0] #tupleです

pprint.pprint(y_all_combination[0:5])
len(y_all_combination)==2**16 #True

[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0)]


True

In [113]:
find_E0_df = pd.DataFrame()
find_E0_df.index.name = 'the number of y which energy is 0'
find_E0_df.columns.name = 'the number of selected features'

In [61]:
def calcu_corr(y):
    return (y-np.mean(y))/(np.sqrt((X.T*X.T).sum(axis=1))-X.shape[0]*(X.mean(axis=0).getA1())**2) 

In [55]:
#エラー発生源調査

def error_happend_function(y):
    y = y - np.mean(y)
    # compute the correlation  
    corr = safe_sparse_dot(y, X)#y, Xが疎行列であることが前提。returnされるのはret=y*Xまたはnp.dot(y, X)
    #pprint.pprint(corr)
    corr /= np.asarray(np.sqrt(safe_sqr(X).sum(axis=0))).ravel()#((Xの2乗)を行ごとにsum)をコピーしたもので割る   
    corr /= norm(y) #corrをyのベクトル長で割る
    #pprint.pprint(corr)
    #pprint.pprint(corr ** 2)
    if (all(1 - corr ** 2 == 0))== False:
        print('1 - corr ** 2 == 0') #全部の要素が1 - corr ** 2 != 0なんて無理だよーということらしい。

In [56]:
for y_new in y_all_combination:
    error_happend_function(y_new) #ほぼ全部false

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 

1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 

1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 

1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0


...


1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr **

1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 

1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 

1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0
1 - corr ** 2 != 0


KeyboardInterrupt: 

In [62]:
for y_new in y_all_combination:
    corr = calcu_corr(y_new)
    if 1 - corr ** 2 == 0:
        print('{}を使うと絶対エラーが出ます'.format(y_new))

AttributeError: 'Series' object has no attribute 'getA1'

In [116]:
E0_count = 0
for y_new in y_all_combination:
    if y_new != (0,) * len(y):
        if y_new != (1,) * len(y):
            y_new = pd.Series(list(y_new))
            #error_happend_function(y_new)
            mask_new = selection(num_select, X, y_new)
            E_new = selected_X_energy - each_energy(mask_new, num_select)
            if E_new == 0:
                E0_count += 1

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/featu

In [ ]:
find_E0_df[num_select] = E0_count

find_E0_df